In [1]:
function error(errstring)
    println(errstring)
    input = readline()
    
end


error (generic function with 1 method)

In [2]:
function swapRows!(A,i,j)
    if ndims(A) == 1
        A[i],A[j] = A[j],A[i]
    else
        A[i,:],A[j,:] = A[j,:],A[i,:]
    end
    return A
end

swapRows! (generic function with 1 method)

### Question 1

In [3]:
using LinearAlgebra
function luPivotDecompose(A::Array{Float64,2}, tol=1.0e-9)
    n = length(A[:,1])
    count = 0 # store the number of swaps
    
    #store the sequence of the permutation of the rows
    seq = collect(1:n)
    
#     set up scale factors
    s = zeros(n)
    for i in 1:n
        s[i] = maximum(abs.(A[i,:]))
    end
    
    
    # Row interchange, if needed
    for k in 1:n-1
        
        p = argmax(abs.(A[k:n,k]) ./ s[k:n]) +k -1
     

        if abs(A[p,k]) < tol
            error("Matrix is singular")
        end
        
        if p != k
            swapRows!(seq,k,p)
            swapRows!(s,k,p)
            swapRows!(A,k,p)
            count += 1
        end   
        
        # Elimination
        for i in k+1:n
            if A[i,k] != 0.0
                lam = A[i,k] / A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - lam * A[k,k+1:n]
                A[i,k] = lam # Lower triangle entry
            end
        end
    end
    
    if abs(A[n,n]) < tol
        error("Matrix is singular")
    end
    
    return A,seq,count
    
end



luPivotDecompose (generic function with 2 methods)

### Question 2

In [4]:
function luPivotSolve(A::Array{Float64,2}, b::Array{Float64,1}, seq::Array{Int64,1})
    n = length(A[:,1])
    
    # Rearrange constant vector; store it in [x]
    x = copy(b)
    for i in 1:n
        x[i] = b[seq[i]]
    end
    
    # Solution
    #Forward substitution
    for k in 2:n
        x[k] = x[k] - LinearAlgebra.dot(A[k,1:k-1],x[1:k-1])
    end
    
    # Back substitution
    x[n] = x[n]/A[n,n]
    for k in n-1:-1:1
        x[k] = (x[k] - dot(A[k,k+1:n],x[k+1:n]))/A[k,k]
    end
    
    return x
end

luPivotSolve (generic function with 1 method)

### Question 3 : Compare methods with large random matrices and vectors

In [5]:
matrix_a = rand(1000,1000)

1000×1000 Array{Float64,2}:
 0.0605924  0.0669169  0.94833    …  0.414362    0.114062   0.480882  
 0.341081   0.116668   0.137152      0.46496     0.0640719  0.248693  
 0.815593   0.0859542  0.226785      0.394851    0.40925    0.775817  
 0.19725    0.867762   0.13232       0.74098     0.130002   0.051102  
 0.249262   0.999397   0.597381      0.926297    0.815777   0.165565  
 0.730319   0.928283   0.801096   …  0.608958    0.680293   0.200707  
 0.253218   0.540924   0.767222      0.912933    0.532453   0.251979  
 0.127071   0.79517    0.217983      0.24927     0.299723   0.411906  
 0.524867   0.905576   0.228524      0.516371    0.38944    0.518536  
 0.842023   0.294558   0.265709      0.00374739  0.0504173  0.760731  
 0.281455   0.270139   0.728097   …  0.268755    0.25914    0.555953  
 0.844446   0.414754   0.0371038     0.0186595   0.114925   0.00615799
 0.751466   0.790342   0.717775      0.842821    0.350749   0.873764  
 ⋮                                ⋱              

##### original_matrix will be used by left division \ of Julia method

In [6]:
original_matrix = copy(matrix_a)

1000×1000 Array{Float64,2}:
 0.0605924  0.0669169  0.94833    …  0.414362    0.114062   0.480882  
 0.341081   0.116668   0.137152      0.46496     0.0640719  0.248693  
 0.815593   0.0859542  0.226785      0.394851    0.40925    0.775817  
 0.19725    0.867762   0.13232       0.74098     0.130002   0.051102  
 0.249262   0.999397   0.597381      0.926297    0.815777   0.165565  
 0.730319   0.928283   0.801096   …  0.608958    0.680293   0.200707  
 0.253218   0.540924   0.767222      0.912933    0.532453   0.251979  
 0.127071   0.79517    0.217983      0.24927     0.299723   0.411906  
 0.524867   0.905576   0.228524      0.516371    0.38944    0.518536  
 0.842023   0.294558   0.265709      0.00374739  0.0504173  0.760731  
 0.281455   0.270139   0.728097   …  0.268755    0.25914    0.555953  
 0.844446   0.414754   0.0371038     0.0186595   0.114925   0.00615799
 0.751466   0.790342   0.717775      0.842821    0.350749   0.873764  
 ⋮                                ⋱              

In [7]:
vector_b = rand(1000)

1000-element Array{Float64,1}:
 0.538038343573217   
 0.8281423038141411  
 0.42769561049940674 
 0.5787672256266296  
 0.5209461473186967  
 0.8246947755523861  
 0.08776817761541889 
 0.6703820732114947  
 0.8571021365879554  
 0.09207559255489373 
 0.9015637313954581  
 0.25788387134328117 
 0.7464196530244696  
 ⋮                   
 0.4231893422930817  
 0.06921288731502062 
 0.7292763049705191  
 0.026520427780990508
 0.1693194719793394  
 0.4585882013043936  
 0.520486813158676   
 0.6545926192511731  
 0.970708792938549   
 0.8378840078533776  
 0.326728625846719   
 0.6288528118602978  

#### left division \ of Julia outputs,

In [8]:
@timev original_matrix \ vector_b

  1.117274 seconds (3.88 M allocations: 195.052 MiB, 8.98% gc time)
elapsed time (ns): 1117273864
gc time (ns):      100342544
bytes allocated:   204526867
pool allocs:       3878830
non-pool GC allocs:730
malloc() calls:    2
GC pauses:         9
full collections:  1


1000-element Array{Float64,1}:
 -0.5653400137791599 
  2.675428884082731  
 -1.3236915501569355 
  2.219771864317339  
 -0.7766467519144182 
  1.7682696918023584 
 -0.5429135994720874 
  2.344787000364937  
 -1.028706531572718  
 -0.9927269001136549 
  2.0524810968514506 
 -1.663848461364546  
 -0.5979097132200833 
  ⋮                  
 -0.871580480846344  
  0.09189021661637485
 -1.6190729657273955 
  1.1845160546385638 
 -0.08137400532450671
 -0.4907170617113744 
 -0.7796153777428771 
 -0.5057267258144883 
  2.157775606411845  
  0.05259217174482376
  1.7026203846277552 
 -0.4438524289104407 

## test and time the function of luPivotDecompose and luPivotSolve 

### First Comparision

In [9]:
@timev M,seq,swaps = luPivotDecompose(matrix_a)

  3.940853 seconds (3.73 M allocations: 10.333 GiB, 21.40% gc time)
elapsed time (ns): 3940853009
gc time (ns):      843170223
bytes allocated:   11095012350
pool allocs:       1841508
non-pool GC allocs:1884958
GC pauses:         484


([0.9950540439624183 0.08920032260477861 … 0.747382052466141 0.8473448483879504; 0.14702771266313255 0.9862964500070758 … 0.39601733968605246 0.3750265751610564; … ; 0.8044779173976688 0.3601248124557466 … 0.2629271897605229 4.607316593532188; 0.8572640756406805 0.37827902455426393 … 0.2521230741354221 2.42378220549295], [851, 492, 1, 670, 962, 958, 917, 30, 534, 264  …  545, 827, 131, 812, 716, 374, 742, 807, 801, 692], 994)

In [10]:
@timev luPivotSolve(M,vector_b,seq)

  0.082095 seconds (155.99 k allocations: 23.455 MiB, 8.24% gc time)
elapsed time (ns): 82095454
gc time (ns):      6763369
bytes allocated:   24594806
pool allocs:       152931
non-pool GC allocs:3056
GC pauses:         1


1000-element Array{Float64,1}:
 -0.5653400137734165 
  2.675428884053163  
 -1.3236915501401103 
  2.2197718642862623 
 -0.7766467519105674 
  1.7682696917911538 
 -0.5429135994624232 
  2.3447870003405646 
 -1.0287065315574766 
 -0.9927269001064436 
  2.052481096829756  
 -1.6638484613569704 
 -0.5979097132120889 
  ⋮                  
 -0.8715804808433788 
  0.09189021662020254
 -1.6190729657070666 
  1.184516054628251  
 -0.08137400533279138
 -0.49071706170407026
 -0.7796153777234731 
 -0.5057267257921866 
  2.157775606389848  
  0.05259217174624653
  1.7026203846078052 
 -0.44385242890929766

### Scond Comparision

In [11]:
vector_b2 = rand(1000)

1000-element Array{Float64,1}:
 0.23358070394993824 
 0.22807633501268976 
 0.9952981719427152  
 0.20774873298308583 
 0.8318234240255791  
 0.9592318458375226  
 0.8940150610391406  
 0.10155888472500774 
 0.24644238669621488 
 0.15779305570098945 
 0.7144027904078731  
 0.18792699558199377 
 0.01904637556714639 
 ⋮                   
 0.52463018370925    
 0.06625797095009633 
 0.255743703441206   
 0.7377625742922584  
 0.04326059012631078 
 0.8099299819572774  
 0.3009957966729322  
 0.033314033941038934
 0.8087379345613932  
 0.567722748123825   
 0.2011184466573428  
 0.40463150705956763 

In [12]:
@timev original_matrix \ vector_b2

  0.033765 seconds (9 allocations: 7.645 MiB, 18.07% gc time)
elapsed time (ns): 33765248
gc time (ns):      6101246
bytes allocated:   8016528
pool allocs:       6
non-pool GC allocs:2
malloc() calls:    1
GC pauses:         1


1000-element Array{Float64,1}:
   9.843044361214112 
 -54.62958430080234  
  30.271277799863412 
 -56.04912766403708  
   7.0959109299670935
 -19.771966504781922 
  18.22904669015719  
 -44.38813892910057  
  27.329050709547673 
  13.8085444626789   
 -39.761464272136216 
  13.170881081885533 
  14.463229828672562 
   ⋮                 
   5.102772480116919 
   7.555867732116547 
  36.38978315808972  
 -18.26858872157535  
 -15.148950385884707 
  13.44618309848701  
  36.32153444691955  
  41.05109415852428  
 -40.82004123121149  
   3.6105809986481825
 -35.7227369092701   
   2.441544816170492 

In [13]:
@timev luPivotSolve(M,vector_b2,seq)

  0.005622 seconds (4.00 k allocations: 15.826 MiB)
elapsed time (ns): 5622384
bytes allocated:   16595232
pool allocs:       980
non-pool GC allocs:3022


1000-element Array{Float64,1}:
   9.843044361111952 
 -54.62958430027231  
  30.271277799561823 
 -56.04912766348316  
   7.095910929896066 
 -19.77196650458495  
  18.229046689983225 
 -44.38813892866666  
  27.329050709274846 
  13.808544462552842 
 -39.761464271751116 
  13.170881081749698 
  14.46322982852768  
   ⋮                 
   5.102772480063737 
   7.555867732044175 
  36.38978315772591  
 -18.268588721392696 
 -15.148950385736352 
  13.446183098357842 
  36.32153444656727  
  41.051094158123334 
 -40.820041230818674 
   3.610580998620182 
 -35.722736908910974 
   2.4415448161515094

   ### Conclusion: left division \ of julia is much faster than the total time of luPivotDecompose and luPivotSolve. But if we have got the LU matrix and sequence of the permutation of the rows. Then we could use the luPivotSolve function to get the x faster than left division \ of julia with different vector_b. Thus the elimination phase take up most of the time in the process.